In [ ]:
!pip install pandas
!pip install networkx
!pip install scipy 

In [ ]:
import pandas as pd
import json
import networkx as nx
G = nx.Graph()

with open('systems.json') as f:  
    data = json.load(f)
df=pd.DataFrame(data)

with open('planets.json') as f:  
    data = json.load(f)
df_planets=pd.DataFrame(data)
#print(df)

#for row in range(len(df)):
#  print(df.at[row, 'Name'])

In [ ]:
#df.dtypes
#df[1:4]
df

In [ ]:
df_planets

In [ ]:
id_to_name = {}
for index, row in df.iterrows():
    id_to_name[row["SystemId"]] = row["Name"]

In [ ]:
G = nx.Graph()
#G.from_pandas_dataframe(df)
#list(G.nodes)

ANT_node = []
BEN_node = []
MOR_node = []
for index, row in df.iterrows():
    G.add_node(row["Name"])
    if row["Name"] == "Antares I":
        ANT_node = row["Name"]
    if row["Name"] == "Benten":
        BEN_node = row["Name"]
    if row["Name"] == "Moria":
        MOR_node = row["Name"]
#    G.add_node(row.to_dict)

for index, row in df.iterrows():
    for connection in row["Connections"]:
        #print(row["Name"] + " to " + id_to_name[connection["ConnectingId"]])
        G.add_edge(row["Name"], id_to_name[connection["ConnectingId"]])
    
print("Node count " + str(G.number_of_nodes()) + " edge count: " + str(G.number_of_edges()))
print(favoredNode)

# maybe we are only interested in the systems near ANT

# within N jumps of ANT
def filterPlanetFunction(systemName):
    pathLen = nx.shortest_path_length(G, ANT_node, systemName)
    # print("Distance from " + systemName + " is " + str(pathLen))
    return pathLen < 15

# planets closer to ANT than they are to other CXes
def filterPlanetFunction(systemName):
    distANT = nx.shortest_path_length(G, ANT_node, systemName)
    distBEN = nx.shortest_path_length(G, BEN_node, systemName)
    distMOR = nx.shortest_path_length(G, MOR_node, systemName)
    # print("Distance from " + systemName + " is " + str(pathLen))
    return distANT <= distBEN and distANT <= distMOR

# all planets
def filterPlanetFunction(systemName):
    return True


selected_nodes = []
for index, row in df.iterrows():
    
    systemName = row["Name"]
    
    if(filterPlanetFunction(systemName)):
        selected_nodes.append(systemName)
        
print("~~~~~~~~~~~~~~~~~~~")
G = G.subgraph(selected_nodes)
print("after filter: Node count " + str(G.number_of_nodes()) + " edge count: " + str(G.number_of_edges()))

In [ ]:
# find game specified xy positions:
game_pos = {}
for index, row in df.iterrows():
    if row["Name"] in G.nodes():
#         print("Find pos for " + row["Name"])
        game_pos[row["Name"]] = [row["PositionX"] / 1000.0 , row["PositionY"] / 1000.0]

In [ ]:
# find systems with nice planets.
filteredValue = {}
filteredValue2 = {}
filteredValue3 = {}

for index, row in df_planets.iterrows():
    
    systemName = id_to_name[row["SystemId"]]
    planetName = row["PlanetName"]
    
    if systemName in G.nodes():

        filteredValue.setdefault(systemName, 0)
        filteredValue2.setdefault(systemName, 0)
        filteredValue3.setdefault(systemName, 0)

    #     if(row["Fertility"] > -1):
    #         print("Fertile: " + str(row["Fertility"]) + " on " + planetName + " in " + systemName)
    #         filteredValue[systemName] = filteredValue.get(systemName, 0) + 1

        malusCount = 0
        pressure = row["Pressure"]
        temperature = row["Temperature"]
        gravity = row["Gravity"]

        if(gravity > 2.5 or gravity < 0.25):
            malusCount += 1
        if(temperature > 75 or temperature < -25):
            malusCount += 1
        if(pressure > 2.0 or pressure < 0.25 ): # low pressure is fine, so or pressure < 0.25 is okay!
            malusCount += 1

        if(malusCount == 0):
            filteredValue[systemName] = filteredValue.get(systemName, 0) + 1


        #filteredValue = dict(nx.single_source_shortest_path_length(G, favoredNode))


        for reso in row["Resources"]:
            #water
            if reso["MaterialId"] == "ec8dbb1d3f51d89c61b6f58fdd64a7f0" \
                and reso["ResourceType"] == "LIQUID" \
                and reso["Factor"] > 0.36: 
                # ResourceType = LIQUID Factor > 0.36 (Harmonia levels)
                filteredValue2[systemName] = 1


        #consider it occupied if it has a COGC
    #     print(row["HasChamberOfCommerce"])
        if row["HasChamberOfCommerce"]:
            filteredValue3[systemName] = 1


In [ ]:
import matplotlib.pyplot as plt

# pos = nx.spring_layout(G, iterations=100, seed=173, scale = 2)
# pos = nx.spiral_layout(G)
# pos = nx.shell_layout(G)
#pos = game_pos
pos = nx.kamada_kawai_layout(G, pos=game_pos)

plt.clf()
plt.figure(figsize=(35,35))

print("Drawing: Node count " + str(G.number_of_nodes()) + " edge count: " + str(G.number_of_edges()))

# draw extra nodes to create outlines and dots inside our real nodes.
nx.draw_networkx_nodes(G, pos, node_size=210, node_color='#00FF00', nodelist=list(filteredValue3.keys()), alpha=list(filteredValue3.values()))

nx.draw(G, pos, node_size=150, with_labels=False, nodelist=list(filteredValue.keys()), node_color=list(filteredValue.values()), cmap="inferno")

nx.draw_networkx_nodes(G, pos, node_size=20, node_color='#0000FF', nodelist=list(filteredValue2.keys()), alpha=list(filteredValue2.values()))


label_ratio = 1.0/16.0
pos_labels = {}
for aNode in G.nodes():
    #Get the node's position from the layout
    x,y = pos[aNode]
    #Get the node's neighbourhood
    N = G[aNode]
    #Find the centroid of the neighbourhood. The centroid is the average of the Neighbourhood's node's x and y coordinates respectively.
    #Please note: This could be optimised further
    cx = sum(map(lambda x:pos[x][0], N)) / len(pos)
    cy = sum(map(lambda x:pos[x][1], N)) / len(pos)
    #Get the centroid's 'direction' or 'slope'. That is, the direction TOWARDS the centroid FROM aNode.
    slopeY = (y-cy)
    slopeX = (x-cx)
    #Position the label at some distance along this line. Here, the label is positioned at about 1/8th of the distance.
    pos_labels[aNode] = [x+slopeX*label_ratio, y+slopeY*label_ratio]
    
    #ignore all the math we did above and just offset it from the planet. 
    pos_labels[aNode] = [x + 0.01, y + 0.01]


# pos = pos_labels
#  "ec": "k",
label_options = { "fc": "white", "alpha": 0.2}
nx.draw_networkx_labels(G, pos_labels, font_size=7, bbox=label_options)

s = "Green outline indicates COGC. Blue dot indicates usable quantities of water ( liquid and > Harmonia ). Color indicates number of planets that don't need a building material other than SEA or MCG"
plt.title("Prosperous Universe\nmap by Archiel\n" + s)
# plt.text(0.5, 0.98, s, horizontalalignment="center")
plt.tight_layout()
plt.show()

